# Working with Figures

All methods return a [Plotly Figure](https://plotly.com/python/figure-structure/) that can be modified and exported.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import xarray as xr

from xarray_plotly import config, xpx

config.notebook()

In [ ]:
df = px.data.stocks().set_index("date")
df.index = df.index.astype("datetime64[ns]")

stocks = xr.DataArray(
    df.values,
    dims=["date", "company"],
    coords={"date": df.index, "company": df.columns.tolist()},
    name="price",
)

## update_layout

Modify [layout properties](https://plotly.com/python/reference/layout/): title, legend, margins, fonts.

In [ ]:
fig = xpx(stocks).line()
fig.update_layout(
    title={"text": "Stock Prices", "x": 0.5},
    legend={"orientation": "h", "y": 1.02},
)
fig

## update_traces

Modify [trace properties](https://plotly.com/python/reference/): line width, markers, opacity.

In [ ]:
fig = xpx(stocks).line()
fig.update_traces(line={"width": 3})
fig

In [ ]:
fig = xpx(stocks).scatter()
fig.update_traces(marker={"size": 10, "opacity": 0.7})
fig

## update_xaxes / update_yaxes

Modify [axis properties](https://plotly.com/python/axes/).

In [ ]:
fig = xpx(stocks).line()
fig.update_xaxes(rangeslider_visible=True)
fig.update_yaxes(tickformat=".0%")
fig

## add_hline / add_vline

Add reference lines. See [shapes](https://plotly.com/python/shapes/).

In [ ]:
fig = xpx(stocks).line()
fig.add_hline(y=1.0, line_dash="dash", line_color="gray", annotation_text="Baseline")
fig.add_vline(x="2018-10-01", line_dash="dot", line_color="red")
fig

## add_annotation

Add text annotations. See [annotations](https://plotly.com/python/text-and-annotations/).

In [ ]:
fig = xpx(stocks).line()
fig.add_annotation(x="2018-10-01", y=1.4, text="Peak", showarrow=True, arrowhead=2)
fig

## add_trace

Add custom traces using [Graph Objects](https://plotly.com/python/graph-objects/).

In [ ]:
fig = xpx(stocks.sel(company="GOOG")).line()

# Add moving average
goog = stocks.sel(company="GOOG")
ma = goog.rolling(date=20, center=True).mean()

fig.add_trace(
    go.Scatter(
        x=ma.coords["date"].values,
        y=ma.values,
        mode="lines",
        name="20-day MA",
        line={"dash": "dash", "color": "red"},
    )
)
fig

## Export

### HTML

See [HTML export](https://plotly.com/python/interactive-html-export/).

```python
fig.write_html("plot.html")
fig.write_html("plot.html", include_plotlyjs="cdn")  # smaller file
```

### Images

Requires [kaleido](https://github.com/plotly/Kaleido): `pip install kaleido`

See [static image export](https://plotly.com/python/static-image-export/).

```python
fig.write_image("plot.png")
fig.write_image("plot.png", scale=2)  # higher resolution
fig.write_image("plot.svg")
fig.write_image("plot.pdf")
```